In [15]:
#Import necessary libraries
import pandas as pd
import requests
import re
import unicodedata
from pymongo import MongoClient
from tqdm import tqdm

In [17]:
#-------------------------------
# CONFIGURATION SECTION
#-------------------------------
MONGO_URI = 'mongodb://localhost:27017/'
DB_NAME = 'games_database'
POSTS_COLLECTION_NAME = 'games_posts_comments'
COMMENTS_COLLECTION_NAME = 'games_posts_comments'

OLAMA_ENDPOINT = 'http://localhost:11434/api/embeddings'
EMBED_MODEL = 'nomic-embed-text'

In [4]:
#Import langchain and ollama
!pip install langchain
!pip install langchain-ollama

In [29]:
#Import necessary libraries
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

#Initialize Ollama LLM
llm = OllamaLLM(model="mistral-nemo")

#Create the prompt
filter_prompt = PromptTemplate.from_template(
    """
    Given the following text, determine if it is relevant to a game review.
    If relevant, return 'KEEP'. If irrelevant, return 'REMOVE'.

    Text: {text}
    """
)

#Use RunnableSequence
filter_chain = filter_prompt | llm

In [31]:
# Import necessary libraries
import re
import unicodedata

#-------------------------------
# FUNCTION DEFINITIONS
#-------------------------------

def parse_timestamp(epoch_float):
    if pd.isna(epoch_float):
        return None
    return datetime.fromtimestamp(float(epoch_float), tz=timezone.utc)
    
def clean_text(text):
    if text is None:
        return None
    # Normalize Unicode to ASCII
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', ' ', text)
    # Remove all characters except a-z and spaces
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Remove all numbers
    text = re.sub(r'\d+', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Limit to first 500 words (if needed)
    words = text.strip().split()
    if len(words) > 500:
        words = words[:500]
    return ' '.join(words)


In [33]:
#-------------------------------
# READ THE EXCEL FILES
#-------------------------------
posts_df = pd.read_excel('C:/Users/aprna/Desktop/Reddit Scraper/Fin scraped/Final_MD_Posts.xlsx')
comments_df = pd.read_excel('C:/Users/aprna/Desktop/Reddit Scraper/Fin scraped/Final_MD_Comments.xlsx')

posts_df = posts_df.where(pd.notnull(posts_df), None)
comments_df = comments_df.where(pd.notnull(comments_df), None)


In [35]:
#-------------------------------
# CONNECT TO MONGODB
#-------------------------------
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
posts_collection = db[POSTS_COLLECTION_NAME]
comments_collection = db[COMMENTS_COLLECTION_NAME]

posts_collection.delete_many({})
comments_collection.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [37]:
#Import necessary libraries
from datetime import datetime, timezone

#Mistral Nemo will run through all posts and accept of reject them
post_docs = []

for _, row in tqdm(posts_df.iterrows(), total=posts_df.shape[0]):
    cleaned_text = clean_text(row.get('selftext'))

    try:
        # pass 'text' matching prompt
        filter_result = filter_chain.invoke({"text": cleaned_text})

        if isinstance(filter_result, dict):
            filter_result = filter_result.get('text')
        filter_result = filter_result.strip()
    except Exception as e:
        print(f"Error filtering post: {e}")
        filter_result = "REMOVE"

    if filter_result == "KEEP":
        doc = {
            "post_id": row.get('post_id'),
            "title": row.get('title'),
            "selftext": row.get('selftext'),
            "selftext_cleaned": cleaned_text,
            "author": row.get('author'),
            "created_date": parse_timestamp(row.get('created_utc')),
        }
        post_docs.append(doc)

#After processing
if post_docs:
    posts_collection.insert_many(post_docs)
    print(f"✅ Inserted {len(post_docs)} posts into MongoDB.")

    # Show a sample of inserted posts
    sample_posts = posts_collection.find().limit(3)  # change 3 to however many you want
    print("\n📋 Sample Inserted Posts:")
    for post in sample_posts:
        print(post)
else:
    print("⚠️ No posts inserted (no posts passed filtering).")


100%|██████████| 130/130 [42:09<00:00, 19.46s/it] 

✅ Inserted 30 posts into MongoDB.

📋 Sample Inserted Posts:
{'_id': ObjectId('680c3364de6ff9399c3bbb2d'), 'post_id': '1jbg678', 'title': "Should I buy Every Assassin's Creed Game?", 'selftext': "Spring Steam Sale is here and like the whole Assassin's Creed Franchise is on sale for like 85 plus I was thinking of getting Assassin's Creed Shadows as well.\nI have played the Ezio games on the PS3 but not on PC and I haven't played the latest Assassin's Creed games.\nShould I buy everything or just some of them since I plan on buying Assassin's Creed Shadows as well?", 'selftext_cleaned': 'spring steam sale is here and like the whole assassin s creed franchise is on sale for like plus i was thinking of getting assassin s creed shadows as well i have played the ezio games on the ps but not on pc and i haven t played the latest assassin s creed games should i buy everything or just some of them since i plan on buying assassin s creed shadows as well', 'author': 'DRN56', 'created_date': dateti

In [43]:
#Build a DataFrame directly from what was inserted
posts_df_final = pd.DataFrame(post_docs)
print("\n📋 DataFrame Created from Inserted Posts:")
print(posts_df_final.head())


📋 DataFrame Created from Inserted Posts:
   post_id                                              title  \
0  1jbg678          Should I buy Every Assassin's Creed Game?   
1  1ji5ih9  Assassin's Creed Shadows vs Xenoblade Chronicl...   
2  1ji227j  Montser Hunter Wilds, Assassin's Creed Shadows...   
3  1jmc72g  Need recommendations for a linear, non-open wo...   
4  1j634m2  Monster Hunter Wilds has been out for a week n...   

                                            selftext  \
0  Spring Steam Sale is here and like the whole A...   
1  1. Looking for an open-world RPG that emphasis...   
2  Which game should I buy? I can either play it ...   
3  I'm currently playing Assassin's Creed Shadows...   
4  Game: Monster Hunter Wilds\n\nInitial release ...   

                                    selftext_cleaned             author  \
0  spring steam sale is here and like the whole a...              DRN56   
1  looking for an open world rpg that emphasis im...  Marinebiologist_0   
2  wh

In [45]:
#Apply clean_text to 'title' and temporarily store in 'title_cleaned'
posts_df_final['title_cleaned'] = posts_df_final['title'].apply(clean_text)

#Replace 'title' column with the cleaned version
posts_df_final['title'] = posts_df_final['title_cleaned']

#Drop the 'title_cleaned' helper column (no need to save it)
posts_df_final = posts_df_final.drop(columns=['title_cleaned'])

#Save the updated DataFrame to CSV
posts_df_final.to_csv("cleaned_posts_final11.csv", index=False)
print("✅ Cleaned posts saved into 'cleaned_posts_final11.csv' successfully.")

✅ Cleaned posts saved into 'cleaned_posts_final.csv' with 'title' cleaned.


In [ ]:
#-------------------------------
# PROCESS COMMENTS
#-------------------------------
print("Processing comments...")

comment_docs = []

for _, row in tqdm(comments_df.iterrows(), total=comments_df.shape[0]):
    cleaned_text = clean_text(row.get('comment_body'))

    try:
        # Match the new way: pass 'text' key
        filter_result = filter_chain.invoke({"text": cleaned_text})

        if isinstance(filter_result, dict):
            filter_result = filter_result.get('text')
        filter_result = filter_result.strip()
    except Exception as e:
        print(f"Error filtering comment: {e}")
        filter_result = "REMOVE"

    if filter_result == "KEEP":
        doc = {
            "comment_id": row.get('comment_id'),
            "post_id": row.get('post_id'),
            "comment_body": row.get('comment_body'),
            "comment_body_cleaned": cleaned_text,
            "author": row.get('author'),
            "created_date": parse_timestamp(row.get('created_utc')),  # fix here too
            # embedding removed
        }
        comment_docs.append(doc)

#After processing
if comment_docs:
    comments_collection.insert_many(comment_docs)
    print(f"✅ Inserted {len(comment_docs)} comments into MongoDB.")

    # 🔥 Show a sample of inserted comments
    sample_comments = comments_collection.find().limit(3)
    print("\n📋 Sample Inserted Comments:")
    for comment in sample_comments:
        print(comment)
else:
    print("⚠️ No comments inserted (no comments passed filtering).")

print("\nETL Process Completed Successfully!")

Processing comments...


  2%|▏         | 309/12464 [50:57<32:01:37,  9.49s/it] 

In [ ]:
#Save the updated DataFrame to CSV
comments_df_final.to_csv("cleaned_comments_final11.csv", index=False)
print("✅ Cleaned commentss saved into 'cleaned_comments_final11.csv' successfully.")